In [1]:
import requests
import json
from bs4 import BeautifulSoup

### parameter
- seviceKey : id
- target : admrul
- query : 검색어(default: * \)
- numOfRows : 한 페이지 결과수
- pageNo : 페이지 번호

In [2]:
ServiceKey ='qHBjb9yik8lx%2B3bJ3tLeWux4VlZOlRcSe0otBb3Yh4T5c24Yia5vI6e%2BVFFUYHRwWl8FgTAvgRJjWf9XiIlpQg%3D%3D'

In [3]:
url = 'http://apis.data.go.kr/1170000/law/admrulSearchList.do?'
url += 'serviceKey=' + ServiceKey + '&query=*&target=admrul&numOfRows=10&pageNo=1'

In [4]:
res = requests.get(url).text

In [5]:
soup = BeautifulSoup(res,'html.parser')

In [6]:
crawl_ls = soup.select('admrul')
id_ls = []
for crawl in crawl_ls:
    crawl = str(crawl)
    
    start = crawl.index('ID=')
    crawl = crawl[start+3:]
    end = crawl.index('&')
    crawl = crawl[:end]

    id_ls.append(crawl)

In [14]:
params = {}
url_detail = 'http://www.law.go.kr/LSW/admRulAttFlList.do' 
file_ls = []
for admid in id_ls:
    file_info = []
    params.update({'admRulSeq':admid})
    res = requests.post(url_detail, data=params).text
    soup = BeautifulSoup(res,'html.parser')
    files = soup.select('ul li a')
    for file in files:
        file_info.append([file['href'],file.text])
    file_ls.append(file_info)

In [15]:
url_file = 'http://www.law.go.kr/LSW/'
for files in file_ls:
    for file in files:
        print(url_file+file[0])
        res = requests.get(url_file+file[0])
    break

http://www.law.go.kr/LSW/flDownload.do?flSeq=25381477
